In [1]:
import pandas as pd
import numpy as np
import time, gc, math
from pandas.io import gbq
import matplotlib.pyplot as plt
import bs4
import urllib
import requests
import json
import codecs
import binascii
import os
%matplotlib inline

### 使用する関数の定義

In [2]:
#https://www.google.co.jp/search?tbm=isch&q=%E4%BB%8A%E7%94%B0%E7%BE%8E%E6%A1%9C+%E6%A8%AA%E9%A1%94&chips=q:%E4%BB%8A%E7%94%B0%E7%BE%8E%E6%A1%9C+%E6%A8%AA%E9%A1%94,g_1:%E3%82%A2%E3%82%A4%E3%82%B3%E3%83%B3&sa=X&ved=0ahUKEwijn5nqjoXfAhWExrwKHeCqDUMQ4lYILigI&biw=1440&bih=821&dpr=1
def get_images(query, start):
    screen_width = 1920
    screen_height = 1080
    params = {
        "q": query,
        "sa": "X",
        "biw": screen_width,
        "bih": screen_height,
        "tbm": "isch",
        "start": start,
        #"ei": "" - This seems like a unique ID, you might want to use it to avoid getting banned. But you probably still are.
    }
    header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}
    url="https://www.google.co.jp/search"
    
    URL = '{}?{}'.format(url, urllib.parse.urlencode(params))
    print(URL)

    req = urllib.request.Request(URL,headers=header)
    bs = bs4.BeautifulSoup(urllib.request.urlopen(req), 'html.parser')
    
    ActualImages=[]# contains the link for Large original images, type of  image
    cnt = 1
    for a in bs.findAll("div",{"class":"rg_meta"}):
        link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
        ActualImages.append((link,Type))
        
    return ActualImages

### キャスト一覧から画像をダウンロード

In [3]:
#タレント一覧の読み込み
cast_list = pd.read_csv("unique_cast_list.csv",encoding="utf-8-sig")

In [ ]:
#キャスト名を用いて、google画像検索を行い、画像をダウンロード
for idx, row in cast_list.iterrows():
    keyword = row.cast_name
    
    if idx < 999:
        continue
        
    print(idx, keyword) 
    
    
    x = 0
    _query = keyword
    ActualImages = get_images(_query.encode('utf-8'), x*100)
    
    dir_name = _query.replace("/", " ").replace(":", " ")
    
    save_directory =  './talent_image/' + dir_name
    print(save_directory)

    for i , (img , Type) in enumerate( ActualImages):
        
        if i > 5:
            break
        try:
            Type = Type if len(Type) > 0 else 'jpg'
#             print("Downloading image {} ({}), type is {}".format(i, img, Type))
            is_dir = os.path.isdir(save_directory)

            if not is_dir:
#                 print("make directory:", save_directory)
                os.mkdir(save_directory)
                
            img_path = os.path.join(save_directory , "img_"+str(i)+"."+Type)
            
            if os.path.isfile(img_path):
                print(img_path, "already exist")
            else:
                raw_img = urllib.request.urlopen(img, timeout=10).read()
                f = open(img_path, 'wb')
                f.write(raw_img)
                f.close()
        except Exception as e:
            print ("could not load : "+img)
            print (e)

999 宮城夏鈴
https://www.google.co.jp/search?biw=1920&sa=X&start=0&bih=1080&tbm=isch&q=%E5%AE%AE%E5%9F%8E%E5%A4%8F%E9%88%B4
./talent_image/宮城夏鈴
./talent_image/宮城夏鈴/img_0.jpg already exist
./talent_image/宮城夏鈴/img_1.jpg already exist
./talent_image/宮城夏鈴/img_2.jpg already exist
./talent_image/宮城夏鈴/img_3.png already exist
./talent_image/宮城夏鈴/img_4.jpg already exist
./talent_image/宮城夏鈴/img_5.jpg already exist
1000 Reichi
https://www.google.co.jp/search?biw=1920&sa=X&start=0&bih=1080&tbm=isch&q=Reichi
./talent_image/Reichi
./talent_image/Reichi/img_0.jpg already exist
./talent_image/Reichi/img_1.jpg already exist
./talent_image/Reichi/img_2.jpeg already exist
./talent_image/Reichi/img_3.jpg already exist
./talent_image/Reichi/img_4.jpg already exist
./talent_image/Reichi/img_5.png already exist
1001 宮崎あおい
https://www.google.co.jp/search?biw=1920&sa=X&start=0&bih=1080&tbm=isch&q=%E5%AE%AE%E5%B4%8E%E3%81%82%E3%81%8A%E3%81%84
./talent_image/宮崎あおい
1002 ナヲ(マキシマムザホルモン)
https://www.google.co.jp/search?